In [52]:
import base64
from io import BytesIO

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import chi2_contingency

from ydata_profiling import ProfileReport

In [2]:
df_train = pd.read_csv('../datasets/train.csv')
df_val = pd.read_csv('../datasets/val.csv')
df_test = pd.read_csv('../datasets/test.csv')

df_full = pd.concat([df_train, df_val, df_test])
print(f"{df_full.shape=}")
df_full.head()

df_full.shape=(113771, 38)


,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,...,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN,DBWT
0,194,1,1,33.3,0,99,1,2,1428,3,...,3,0,188,1,1,N,0,F,27,3810.0
1,3945,1,1,37.4,0,12,2,9,1645,6,...,0,1,232,1,1,N,0,F,9,3562.0
2,84759,1,1,18.1,0,10,,7,746,7,...,1,0,99,1,1,N,0,F,33,3085.0
3,27656,1,1,23.4,0,99,1,2,1453,2,...,0,0,120,1,1,N,0,F,29,3380.0
4,94701,1,1,25.7,0,11,2,7,645,3,...,0,0,150,1,1,N,0,M,42,3220.0


In [3]:
df_train.describe(include="all")

,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,...,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN,DBWT
count,86465.000000,86465.000000,86465.000000,86465.000000,86465.000000,86465.000000,86465,86465.000000,86465.000000,86465.000000,...,86465.000000,86465.000000,86465.000000,86465.000000,86465.000000,86465,86465.000000,86465,86465.000000,86465.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,M,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,45455,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,72839,NaN,44219,NaN,NaN
mean,54145.562783,1.329821,1.034268,28.854351,1.555392,10.906436,NaN,6.571757,1233.401908,4.057376,...,1.276528,0.705627,176.216724,1.797051,1.330642,NaN,0.261158,NaN,31.525369,3259.964367
std,31247.174131,0.765523,0.314891,12.704882,8.219926,19.742169,NaN,3.418633,632.705612,1.855229,...,3.951516,5.323970,124.797997,1.166763,0.532553,NaN,2.043951,NaN,19.091713,589.164544
min,0.000000,1.000000,1.000000,13.600000,0.000000,1.000000,NaN,1.000000,0.000000,1.000000,...,0.000000,0.000000,75.000000,1.000000,1.000000,NaN,0.000000,NaN,0.000000,227.000000
25%,27037.000000,1.000000,1.000000,22.300000,0.000000,4.000000,NaN,4.000000,800.000000,3.000000,...,0.000000,0.000000,130.000000,1.000000,1.000000,NaN,0.000000,NaN,20.000000,2966.000000
50%,54203.000000,1.000000,1.000000,25.800000,0.000000,7.000000,NaN,7.000000,1238.000000,4.000000,...,1.000000,0.000000,150.000000,1.000000,1.000000,NaN,0.000000,NaN,30.000000,3300.000000
75%,81237.000000,1.000000,1.000000,31.200000,0.000000,10.000000,NaN,10.000000,1736.000000,6.000000,...,2.000000,1.000000,183.000000,3.000000,2.000000,NaN,0.000000,NaN,40.000000,3629.000000


In [4]:
def summarize_column(column):
    return pd.Series([column.nunique(), column.dtype, set(column)], index=['nunique', 'dtype', 'values'])

df_train.apply(summarize_column).T.sort_values(by=['dtype', 'nunique'])

,nunique,dtype,values
NO_RISKS,2,int64,"{0, 1}"
MBSTATE_REC,3,int64,"{1, 2, 3}"
NO_INFEC,3,int64,"{0, 1, 9}"
NO_MMORB,3,int64,"{0, 1, 9}"
RESTATUS,4,int64,"{1, 2, 3, 4}"
PAY_REC,5,int64,"{1, 2, 3, 4, 9}"
RDMETH_REC,5,int64,"{1, 2, 3, 4, 9}"
ATTEND,6,int64,"{1, 2, 3, 4, 5, 9}"
DOB_WK,7,int64,"{1, 2, 3, 4, 5, 6, 7}"
BFACIL,8,int64,"{1, 2, 3, 4, 5, 6, 7, 9}"


**Analyzing the four variables that encode different things `ILLB_R`, `ILOP_R`, `ILP_R`**

Obserations:
- From first category (000-003 Plural delivery) only 3 occurs $\implies$ map to binary vars `ILLB_R_plural`
- Keep second cateogry (004-300 Months since last live birth) as is


Preprocessing strategy:
- One-hot encode into 3 categories ($x=3$, $x=888$, $3<x<888$). The first two categories are binary variables, the third is a continuous variable.
- Missing values are imputed with their most frequent category. For `ILLB_R` and `ILP_R` this is the numerical category, where we impute with the median, and for `ILOP_R` this is category 888 where we simply impute with the one-hot encoding 1.

In [24]:
var = 'ILP_R'
print((df_full[var]<=3).mean())
# df_full[var].value_counts().sort_values(ascending=False)
df_full[var].value_counts().sort_index()

0.011654991166465971


ILP_R
3       1326
4         17
5         29
6         37
7         58
       ...  
295        1
298        1
300        8
888    34891
999    13614
Name: count, Length: 280, dtype: int64

In [62]:
df_full['MEDUC'] = df_full['MEDUC'].astype('object')
df_full.select_dtypes(include=['object']).columns

Index(['DMAR', 'LD_INDL', 'MEDUC', 'RF_CESAR', 'SEX'], dtype='object')

In [26]:
vars = ['ILLB_R', 'ILOP_R', 'ILP_R']
df = df_full[vars]

df_plural = df[df.apply(lambda row: (row==3).any(), axis=1)]

In [33]:
# From first category 000-003, only 3 occurs
df.min()

ILLB_R    3
ILOP_R    3
ILP_R     3
dtype: int64

In [45]:
percent_3 = (df == 3).mean() * 100
percent_between_3_and_888 = ((df > 3) & (df < 888)).mean() * 100
percent_888 = (df == 888).mean() * 100
percent_999 = (df == 999).mean() * 100

percent_df = pd.DataFrame({
    'Percent of category = 3': percent_3,
    'Percent of entries between 3 and 888': percent_between_3_and_888,
    'Percent of category = 888': percent_888,
    'Percent of category = 999': percent_999
}).T

percent_df

,ILLB_R,ILOP_R,ILP_R
Percent of category = 3,1.318438,0.014063,1.165499
Percent of entries between 3 and 888,57.743186,17.509734,56.200614
Percent of category = 888,37.767094,72.566823,30.667745
Percent of category = 999,3.171283,9.909379,11.966143


In [30]:
# ILLB_R and ILP_R are mostly 3 at the same time
(df_plural['ILLB_R'] == df_plural['ILP_R']).mean()

0.8728476821192053

In [32]:
df_plural['ILOP_R'].sort_values()

29021      3
33126      3
81268      3
19760      3
76951      3
        ... 
49062    999
14757    999
36760    999
2108     999
32870    999
Name: ILOP_R, Length: 1510, dtype: int64

#### Report using ydata-profiling

In [3]:
report = ProfileReport(df_train, title='EDA')
report.to_file("../reports/eda/ydata_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# TODO: try reports with the following tools
# - Sweetviz
# - Dtale
# - AutoViz
# - Facets

#### Manual EDA

In [55]:
target = df_train['DBWT']
df = df_train.drop(columns=['id', 'DBWT'])

In [69]:
def make_jointplots(df, target):
    html_content = "<html><head><title>EDA: Jointplots</title></head><body><h1>EDA: Jointplots</h1>"

    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            # Numerical data: jointplot
            correlation = target.corr(df[column])
            jplot = sns.jointplot(x=df[column], y=target, kind="reg")
            title = f"{column} vs. DBWT: Correlation={correlation:.2f}"
        else:
            # Categorical data: violin plot
            # p-value for the Chi-Squared test
            contingency_table = pd.crosstab(df_cat[column], target)
            _, p, _, _ = chi2_contingency(contingency_table)            
            plt.figure(figsize=(10, 6))
            jplot = sns.violinplot(x=df[column], y=target)
            title = f"{column} vs. DBWT: Chi-Squared p-value={p:.5f}"
        plt.tight_layout()

        # Save plot to a temporary buffer
        buffer = BytesIO()
        plt.savefig(buffer, format="png")
        plt.close()
        buffer.seek(0)
        image_png = buffer.getvalue()
        buffer.close()

        # Encode PNG image to base64 string
        image_base64 = base64.b64encode(image_png).decode("utf-8")
        html_content += (
            f'<h3>{title}</h3><img src="data:image/png;base64,{image_base64}"/><br/>'
        )

    html_content += "</body></html>"

    # Save the HTML content to a file
    with open("../reports/eda/jointplots.html", "w") as file:
        file.write(html_content)


make_jointplots(df, target)

In [58]:
correlations = df.select_dtypes(exclude=['object']).corrwith(target)
correlations_abs = correlations.abs().sort_values(ascending=False)

print("Variables sorted by their importance based on absolute correlation with target:")
print(correlations_abs)

Variables sorted by their importance based on absolute correlation with target:
WTGAIN         0.107814
M_Ht_In        0.105115
FAGECOMB       0.090754
NO_RISKS       0.075012
MEDUC          0.074258
ATTEND         0.060404
ILLB_R         0.055659
ILP_R          0.050403
PAY_REC        0.050395
RDMETH_REC     0.047793
CIG_0          0.045028
MAGER          0.042579
PRECARE        0.039942
PAY            0.036701
FEDUC          0.032428
DLMP_MM        0.026314
BFACIL         0.022113
RESTATUS       0.017114
MBSTATE_REC    0.010871
BMI            0.009198
PWgt_R         0.009181
RF_CESARN      0.008000
PRIORTERM      0.006928
PREVIS         0.005309
NO_MMORB       0.004943
ILOP_R         0.004023
PRIORLIVE      0.002891
NO_INFEC       0.002590
DOB_WK         0.002047
DOB_TT         0.001428
DOB_MM         0.000648
PRIORDEAD      0.000391
dtype: float64


In [93]:
max_values = df.max()
print(max_values.value_counts())
print(max_values[max_values == 99.9])
print(max_values[max_values == 99])
print(max_values[max_values == 999])
print(max_values[max_values == 9999])

99      11
9        9
999      4
Y        2
99.9     1
2        1
12       1
9999     1
7        1
50       1
3        1
1        1
4        1
M        1
Name: count, dtype: int64
BMI    99.9
dtype: object
CIG_0        99
DLMP_MM      99
FAGECOMB     99
M_Ht_In      99
PRECARE      99
PREVIS       99
PRIORDEAD    99
PRIORLIVE    99
PRIORTERM    99
RF_CESARN    99
WTGAIN       99
dtype: object
ILLB_R    999
ILOP_R    999
ILP_R     999
PWgt_R    999
dtype: object
DOB_TT    9999
dtype: object
